Fallstudie:<br> 
**Erstellen eines Prognosemodells des Kreditkartenzahlungsverkehr für Online-Einkäufe**

# Import & Helpers

In [6]:
#! mlflow server --host 127.0.0.1 --port 8080

In [7]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.option_context('mode.use_inf_as_na', True)
pd.set_option('display.max_columns', 100)
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

import datetime as dt
import numpy as np
import re

from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set(style='darkgrid',)
color_pal = sns.color_palette("muted")
sns.set_palette(color_pal)
sns.set_context("paper")
%matplotlib inline
# plot dimensions
plot_width = 12
plot_height = 8
palette_success ={0: color_pal[1], 1: color_pal[2]}

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, precision_score, f1_score, recall_score, make_scorer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.inspection import permutation_importance

import mlflow
from mlflow.models import infer_signature

In [8]:
# parameter
rs = 23 # random state

# MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# date of today as string
dt_today = str(dt.date.today())

## Track model in MFflow

In [9]:
def track_model(training_info, model, x_test, y_test, y_pred, y_pred_auc, feature_names):
    """
    Logging all relevant information (metrics, parameters and feature importances) of a trained model to MLflow.
    Model-specific feature importances is used (if existing) and Permutation Feature Importance.
    """
    # Set a tag as detailed description of run
    mlflow.set_tag("Training Info", training_info)
    
    # Log the hyperparameters
    mlflow.log_params(model.get_params())
    
    # Log the metrics
    mlflow.log_metrics({"AUC": roc_auc_score(y_test, y_pred_auc), 
                       "Precision": precision_score(y_test, y_pred, zero_division=np.nan), 
                       "Recall": recall_score(y_test, y_pred, zero_division=np.nan), 
                       "F1-score": f1_score(y_test, y_pred, zero_division=np.nan)})
     
    # check options to find out model-specific FEATURE IMPORTANCES
    plot_title = "No information on feature importances"
    if (hasattr(model, "feature_importances_") == True):
        feat_imp = model.feature_importances_
        plot_title = "Feature importances"
    elif (hasattr(model, "coef_") == True):
        feat_imp = model.coef_[0]
        plot_title = "Coefficients"
    else:
        feat_imp = len(feature_names) * [0]
        print("Has NO feature importances")
    
    # save feature importance to DF
    df_feat_imp = (pd.DataFrame(list(zip(feature_names, feat_imp)), 
                             columns=["Feature", "Model_specific_imp"])
                .set_index("Feature"))
    
    # PERMUTATION FEATURE IMPORTANCE (PFI) 
    pfi_result = permutation_importance(estimator=model, X=x_test, y=y_test, scoring="roc_auc", random_state=rs, n_repeats=10)
    df_pfi = (pd.DataFrame(list(zip(feature_names, pfi_result.importances_mean)), 
                           columns=["Feature", "PFI"])
              .set_index("Feature"))
    # join both dataframes
    df_feat_imp = df_feat_imp.join(df_pfi, how="left").sort_values("PFI", ascending=False).reset_index()
    
    csv_path = "artifacts/feature_importance.csv"
    df_feat_imp.to_csv(csv_path, index=True)
    mlflow.log_artifact(csv_path, "feature_importance.csv")
    
    # Log plot of feature importance
    fig, ax = plt.subplots(1, 2, figsize=(20,10))
    sns.barplot(y="Feature", x="Model_specific_imp", data=df_feat_imp, ax=ax[0])
    ax[0].set_title("Model-specific feature importances", fontsize=10)
    sns.barplot(y="Feature", x="PFI", data=df_feat_imp, ax=ax[1])
    ax[1].set_title("Permutation feature importances", fontsize=10)
    #plt.title(plot_title)
    #plt.xlabel("Importance")
    #plt.ylabel("Feature")
    plt.subplots_adjust(left=0.2, right=0.95, top=0.9, bottom=0.1, wspace=0.6)
    mlflow.log_figure(fig, "feature_importance.png")
    
    
    # Infer the model signature
    #signature = infer_signature(x_train, y_train)
    # Log the model
    """model_info = mlflow.sklearn.log_model(sk_model=model, 
                                          artifact_path="baseline_model", 
                                          signature=signature, 
                                          input_example=x_train.head(5))"""

## Training of model

In [10]:
def train_model(zip_iter, x_train_raw, x_train_numerical, x_test_raw, x_test_numerical, y_train, y_test, features_numerical, exp_name, info_text):
    """
    Iterates over a list of classifiers and parameters to train models. The trained models are tracked in MLflow.
    """
    for n, c, fm in zip_iter:
        print(f"Training and logging of model {n} as classifier {c} with feature mode {fm}")
        if fm == "raw":
            x_train = x_train_raw
            x_test = x_test_raw
            feature_names = list(x_train_raw.columns)
        else:
            x_train = x_train_numerical
            x_test = x_test_numerical
            feature_names = features_numerical
            
        # Create a new MLflow Experiment
        mlflow.set_experiment(exp_name)
        
        # Start an MLflow run
        run_name = n + "_" + exp_name
        with mlflow.start_run(run_name=run_name) as run:
            training_info = n + " - " + info_text
            # Training and prediction
            model = c
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)
    
            if (hasattr(model, "decision_function") == True):
                y_pred_auc = model.decision_function(x_test)
            else:
                y_pred_auc = model.predict_proba(x_test)[:, 1]
    
            track_model(training_info, model, x_test, y_test, y_pred, y_pred_auc, feature_names)
    
            run_id = run.info.run_id
        
        print(f"For {n}: Run logged in MLflow with {run_id}.")
        print("---------------------- \n")

## Hypertune models

In [11]:
def hp_tune_model(search_mode, zip_iter, 
                  x_train_raw, x_train_numerical, 
                  x_test_raw, x_test_numerical, 
                  y_train, y_test, 
                  features_numerical, exp_name, info_text):
    """
    Iterates over a list of classifiers and parameters to train models. The trained models are tracked in MLflow.
    Choose between RandomSearchCV and GridSearchCV.
    search_mode = random/grid
    """
    
    # Config RandomizedSearchCV
    scorer_auc = make_scorer(roc_auc_score, response_method=["predict_proba", "decision_function"])
    scorer_precision = make_scorer(precision_score)
    scorer_recall = make_scorer(recall_score)
    scorer_f1 = make_scorer(f1_score)
    
    cv_scoring = {"auc": scorer_auc, 
                          "precision": scorer_precision, 
                          "recall": scorer_recall, 
                          "F1": scorer_f1}
    randomized_iter = 30
    cv_folds = 5
    rs = 23
    
    for n, c, p, fm in zip_iter:
        print(f"Training and logging of model {n} as classifier {c} with feature mode {fm}")
        if fm == "raw":
            x_train = x_train_raw
            x_test = x_test_raw
            feature_names = list(x_train_raw.columns)
        else:
            x_train = x_train_numerical
            x_test = x_test_numerical
            feature_names = features_numerical

        # Create a new MLflow Experiment
        mlflow.set_experiment(exp_name)
    
        # Start an MLflow run
        run_name = n + "_BEST_" + exp_name
        with mlflow.start_run(run_name=run_name) as run:
            training_info = n + " - " + info_text

            # RandomizedSearchCV or GridSearchCV
            if (search_mode == "random"):
                random_search = RandomizedSearchCV(c, 
                                                   param_distributions=p, 
                                                   n_iter=randomized_iter, 
                                                   verbose=2, 
                                                   cv=cv_folds, 
                                                   scoring=cv_scoring, 
                                                   refit="auc", 
                                                   random_state=rs, 
                                                   error_score="raise")
                # Training and prediction
                random_search.fit(x_train, y_train)
                model_best = random_search.best_estimator_
            else:
                grid_search = GridSearchCV(c, 
                                           param_grid=p, 
                                           scoring=cv_scoring, 
                                           refit="auc", 
                                           cv=cv_folds, 
                                           verbose=2, 
                                           error_score="raise")
                # Training and prediction
                grid_search.fit(x_train, y_train)
                model_best = grid_search.best_estimator_
                
            y_pred = model_best.predict(x_test)

            if (hasattr(model_best, "decision_function") == True):
                y_pred_auc = model_best.decision_function(x_test)
            else:
                y_pred_auc = model_best.predict_proba(x_test)[:, 1]
    
            track_model(training_info, model_best, x_test, y_test, y_pred, y_pred_auc, feature_names)
    
            run_id = run.info.run_id
        
        print(f"For {n}: Run logged in MLflow with {run_id}.")
        print("---------------------- \n")

# Data preparation 2

In [12]:
df_trx = (pd.read_excel("../data/03_interim/df_trx_training.xlsx", sheet_name="df_trx")
          .drop(columns=["Unnamed: 0"])
          .rename(columns={"x_tmsp":"meta_tmsp"}))

In [13]:
# raw x- and y-variables
x_ = [x for x in list(df_trx.columns) if re.match("^x_", x)!=None]
y_ = [y for y in list(df_trx.columns) if re.match("^y_", y)!=None]
meta_ = [m for m in list(df_trx.columns) if m.find("meta_")!=-1]

In [14]:
df_trx[x_].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37612 entries, 0 to 37611
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   x_country                  37612 non-null  object 
 1   x_amount                   37612 non-null  int64  
 2   x_psp                      37612 non-null  object 
 3   x_3d_secured               37612 non-null  int64  
 4   x_card                     37612 non-null  object 
 5   x_tmsp_wd_str              37612 non-null  object 
 6   x_tmsp_daytime             37612 non-null  object 
 7   x_tmsp_h_sin               37612 non-null  float64
 8   x_tmsp_h_cos               37612 non-null  float64
 9   x_tmsp_wd_sin              37612 non-null  float64
 10  x_tmsp_wd_cos              37612 non-null  float64
 11  x_enc_country_Austria      37612 non-null  bool   
 12  x_enc_country_Germany      37612 non-null  bool   
 13  x_enc_country_Switzerland  37612 non-null  boo

Je nach Modell gibt es 2 Features-Sets:<br>

Für Modelle, die numerische Features benötigen: 
- 16 Features
- davon 6 numerische Features, die skaliert werden müssen
- und 10 Features vom Typ Boolean, die aus dem One Hot Encoding entstanden sind und als Integer encoded werden<br>

Für Modelle, bei denen der Typ der Features egal ist:
- 7 Features
- davon 2 numerische Features vom Typ Integer
- und 5 categorial Features vom Typ Object

In [15]:
# categorize features
feat_object_native = list(df_trx[x_].select_dtypes(include="object").columns) # original features of type object
#feat_object_bool_enc = list(df_trx[x_].select_dtypes(include="bool").columns) # encoded object features as bool
feat_num = list(df_trx[x_].select_dtypes(include="int").columns) # numerical features for scaling

## Train-Test-Split

In [16]:
# shuffle and stratify
x_train, x_test, y_train, y_test = train_test_split(df_trx[x_], df_trx[y_], test_size=0.2, random_state=rs, shuffle=True, stratify=df_trx[y_])
y_train= y_train.iloc[:, 0]
y_test= y_test.iloc[:, 0]

In [17]:
train_size = x_train.shape[0]
test_size = x_test.shape[0]
print(f"Trainings data set consists of {train_size} records, while the test data set consists of {test_size} records.")

Trainings data set consists of 30089 records, while the test data set consists of 7523 records.


## Feature Engineering Pipeline

In [18]:
# column transformer for models requiring numerical, scaled input features
coltransformer_4numerical = make_column_transformer(
    (MinMaxScaler(), make_column_selector(dtype_include="number")), 
    (OrdinalEncoder(), make_column_selector(dtype_include="bool"))
)

## Features

In [19]:
# for model with numerical input
coltransformer_4numerical.fit(x_train)
x_train_numerical = coltransformer_4numerical.transform(x_train)
x_test_numerical = coltransformer_4numerical.transform(x_test)

In [20]:
# names of transformed features
transformed_features = coltransformer_4numerical.get_feature_names_out()

In [21]:
# for model with raw input
x_train_raw = x_train[feat_num + feat_object_native]
x_test_raw = x_test[feat_num + feat_object_native]

# Modeling no. 1 

## Test design
Ziele:
- erfolgreiche Transaktionen in Abhängigkeit vom PSP (und den anderen Umständen) erkennen
- positiv = success
- wir müssen dem Kunden einen PSP zuordnen und wollen daher den PSP wählen, der die höchste Erfolgswahrscheinlichkeit hat
- keine PSP zu wählen ist keine Option
  
Vergleich der Modelle:
- Wir sind nicht nur an der Klassifikation an sich interessiert, sondern v.a. auch an der Wahrscheinlichkeit für einen Erfolg abhängig von dem PSP. Der Predictor muss keine fixe Vorhersage über Erfolg/Misserfolg treffen, sondern in Abhängigkeit vom PSP die Erfolgswahrscheinlichkeit berechnen (es wird kein Threshold festgelegt).
- AUC ist eine Vergleichsmetrik, die unabhängig vom gewählten Threshold ist.

Weitere Metriken zur Bewertung der Performance, aber nicht entscheidend:
- Precision meiner Vorhersage: keine Falschpositiven, um mehrfache Transaktionsgebühren zu sparen
- Recall/TPR/Sensitivity: möglichst viele erfolgreiche Transaktionen vorhersagen, um eine Auswahl an PSPs zu haben, um die Kosten minimieren zu können
- F1-Score als harmonisches Mittel

## Baseline model
2 Strategien kommen in Frage:
- most_frequent: macht wenig Sinn, weil dann kein Erfolg vorhergesagt wird, so dass Precision nicht berechnet werden kann. Recall/Sensitity ist ebenfalls = 0
- stratified: It generates predictions by respecting the class distribution of the training data. It is different from the “most frequent” strategy as it instead associates a probability with each data point of being the most frequent class label.

In [22]:
# Training and prediction
baseline_clf = DummyClassifier(strategy="stratified", random_state=rs)
baseline_clf.fit(x_train_raw, y_train)

y_pred_baseline = baseline_clf.predict(x_test_raw)
y_pred_baseline_prob = baseline_clf.predict_proba(x_test_raw)
auc = roc_auc_score(y_test, y_pred_baseline_prob[:, 1], average="macro")
prec = precision_score(y_test, y_pred_baseline, zero_division=np.nan)
recall = recall_score(y_test, y_pred_baseline, zero_division=np.nan)
f1 = f1_score(y_test, y_pred_baseline, zero_division=np.nan)

print(f"Baseline-model")
print(f"with AUC: {auc:.2f}")
print(f"with precision: {prec:.2f}")
print(f"with recall: {recall:.2f}")
print(f"and F1-score: {f1:.2f}")

Baseline-model
with AUC: 0.50
with precision: 0.21
with recall: 0.21
and F1-score: 0.21


## 1st training of models (w/o class_weights)
https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression<br>
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html#sklearn.naive_bayes.CategoricalNB<br>
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier<br>
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn-tree-decisiontreeclassifier<br>
https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC<br>
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC<br>
https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis<br>
https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis<br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn-ensemble-randomforestclassifier<br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn-ensemble-gradientboostingclassifier<br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier

In [23]:
class_weight = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)

In [24]:
# collect all models
# names
first_step_clf_names = ["Baseline model stratified", 
                        "Logistic Regression", 
                        "Naive Bayes", 
                        "KNN", 
                        "Decision Tree", 
                        "Linear SVM",  
                        "SVC", 
                        "LDA", 
                        "QDA",
                        "Random Forest", 
                        "Gradient Boosting", 
                        "Extra Trees"]

# configure random see
params_rs = {"random_state": rs}

# sklearn models
first_step_clf = [DummyClassifier(strategy="stratified", **params_rs), 
                  LogisticRegression(penalty=None, **params_rs), 
                  CategoricalNB(), 
                  KNeighborsClassifier(), 
                  DecisionTreeClassifier(**params_rs),
                  LinearSVC(**params_rs),
                  SVC(probability=True, **params_rs), 
                  LinearDiscriminantAnalysis(), 
                  QuadraticDiscriminantAnalysis(),
                  RandomForestClassifier(**params_rs), 
                  GradientBoostingClassifier(**params_rs), 
                  ExtraTreesClassifier(**params_rs)]

# feature mode for model
first_step_feat_mode = ["raw", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric"]

first_step_zip = zip(first_step_clf_names, first_step_clf, first_step_feat_mode)

In [25]:
first_exp = "PSP_1a_w/o_class_weights"
first_info = "1. Training for comparison of models WITHOUT hyperparameter tuning"
"""train_model(zip_iter=first_step_zip, 
            x_train_raw=x_train_raw, x_train_numerical=x_train_numerical, 
            x_test_raw=x_test_raw, x_test_numerical=x_test_numerical, 
            y_train=y_train, y_test=y_test,
            features_numerical=transformed_features, 
            exp_name=first_exp, info_text=first_info)"""

'train_model(zip_iter=first_step_zip, \n            x_train_raw=x_train_raw, x_train_numerical=x_train_numerical, \n            x_test_raw=x_test_raw, x_test_numerical=x_test_numerical, \n            y_train=y_train, y_test=y_test,\n            features_numerical=transformed_features, \n            exp_name=first_exp, info_text=first_info)'

## Evaluate & understand 1st model training
Models with lowest performance:
- Decision Tree
- SVC

Features with lowest performance:
- Country: Austria, Germany, Switzerland
- Working day

# Modeling no. 2: Remove least important features and models with lowest performance

## Drop features

In [26]:
# LIGHT set for raw features
x_train_raw_light = x_train_raw.drop(columns=["x_tmsp_wd_str", "x_country"])
x_test_raw_light = x_test_raw.drop(columns=["x_tmsp_wd_str", "x_country"])

In [27]:
# columns to remove in numerical input
index_to_drop = [list(transformed_features).index(i) for i in transformed_features if (i.find("_country_")!=-1) or (i.find("_wd_")!=-1)]

# LIGHT set for model with numerical input
features_light = np.delete(transformed_features, index_to_drop, 0)
x_train_numerical_light = np.delete(x_train_numerical, index_to_drop, 1)
x_test_numerical_light = np.delete(x_test_numerical, index_to_drop, 1)

## Training of models

In [28]:
# collect all models
# names
second_step_clf_names = ["Baseline model stratified", 
                        "Logistic Regression", 
                        "Naive Bayes", 
                        "KNN", 
                        "Linear SVM",   
                        "LDA", 
                        "QDA",
                        "Random Forest", 
                        "Gradient Boosting", 
                        "Extra Trees"]

# sklearn models
second_step_clf = [DummyClassifier(strategy="stratified", **params_rs), 
                  LogisticRegression(penalty=None, **params_rs), 
                  CategoricalNB(), 
                  KNeighborsClassifier(), 
                  LinearSVC(**params_rs), 
                  LinearDiscriminantAnalysis(), 
                  QuadraticDiscriminantAnalysis(),
                  RandomForestClassifier(**params_rs), 
                  GradientBoostingClassifier(**params_rs), 
                  ExtraTreesClassifier(**params_rs)]

# feature mode for model
second_step_feat_mode = ["raw", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric", 
                        "numeric"]

second_step_zip = zip(second_step_clf_names, second_step_clf, second_step_feat_mode)
print(f"length of clf_names {len(second_step_clf_names)} & clf {len(second_step_clf)} & feat_mode {len(second_step_feat_mode)}")

length of clf_names 10 & clf 10 & feat_mode 10


In [29]:
second_exp = "PSP_2a_w/o_class_weights"
second_info = "2. Training of models w/ REDUCED features set &  WITHOUT hyperparameter tuning"
"""train_model(zip_iter=second_step_zip, 
            x_train_raw=x_train_raw_light, x_train_numerical=x_train_numerical_light, 
            x_test_raw=x_test_raw_light, x_test_numerical=x_test_numerical_light, 
            y_train=y_train, y_test=y_test,
            features_numerical=features_light, 
            exp_name=second_exp, info_text=second_info)"""

'train_model(zip_iter=second_step_zip, \n            x_train_raw=x_train_raw_light, x_train_numerical=x_train_numerical_light, \n            x_test_raw=x_test_raw_light, x_test_numerical=x_test_numerical_light, \n            y_train=y_train, y_test=y_test,\n            features_numerical=features_light, \n            exp_name=second_exp, info_text=second_info)'

# Modeling no. 3: Random search for hyperparameter tuning

In [30]:
# collect all models
# names
third_clf_names = ["Logistic Regression", 
                "KNN", 
                "Linear SVM",   
                "LDA", 
                "QDA",
                "Random Forest", 
                "Gradient Boosting", 
                "Extra Trees"]

# configure parameters per model
hp_baseline = {"strategy": ["most_frequent", "stratified"]}
hp_qda = {"reg_param": np.arange(0, 1.01, 0.1)}
hp_lda = {"shrinkage": [None, "auto"]}
hp_lr = {"penalty": ["l1", "l2", "elasticnet", None], 
         "C": np.arange(0.01, 10, 0.1), 
         "l1_ratio": np.arange(0.1, 1.01, 0.1)}
hp_nb = {"alpha": np.arange(0, 10, 0.1)}
hp_knn = {"n_neighbors": np.arange(5, 75, 2), 
          "weights": ["uniform", "distance"], 
          "algorithm": ["ball_tree", "kd_tree", "brute"], 
          "p": np.arange(1, 3.0, 0.1)}
hp_dt = {"criterion": ["gini", "entropy", "log_loss"], 
         "splitter": ["best", "random"], 
         "max_depth": np.arange(2, 52, 2), 
         "min_samples_split": np.arange(2, 52, 2), 
         "min_samples_leaf": np.arange(1, 32, 2)}
hp_lin_svm = {"penalty": ["l1", "l2"], 
              "C": np.arange(0, 15, 0.5)}
hp_svc = {"C": np.arange(0, 10, 0.1), 
          "kernel": ["linear", "poly", "rbf", "sigmoid"], 
          "degree":[2, 3, 4, 5], 
          "gamma":["scale", "auto"]}
hp_rf = {"n_estimators": np.arange(10, 300, 20), 
         "criterion": ["gini", "entropy", "log_loss"], 
         "max_depth": np.arange(2, 60, 2), 
         "min_samples_split": np.arange(2, 50, 2), 
         "min_samples_leaf": np.arange(1, 30, 2), 
         "max_features": [None, "sqrt", "log2"]}
hp_grad_boost = {"loss": ["log_loss", "exponential"], 
                 "learning_rate":np.arange(0.1, 1.5, 0.1), 
                 "subsample": np.arange(0.3, 0.99, 0.1), 
                 "criterion": ["friedman_mse", "squared_error"], 
                 "min_samples_split": np.arange(2, 50, 2), 
                 "min_samples_leaf": np.arange(1, 30, 2), 
                 "max_depth": np.arange(2, 20, 1), 
                 "max_features": [None, "sqrt", "log2"]}
hp_extra = {"criterion": ["gini", "entropy", "log_loss"], 
            "max_depth": np.arange(2, 60, 2), 
            "min_samples_split": np.arange(2, 52, 2), 
            "min_samples_leaf": np.arange(1, 32, 2), 
            "max_features": [None, "sqrt", "log2"], 
            "min_weight_fraction_leaf": np.arange(0, 0.5, 0.1), 
            "bootstrap": [True, False]}

# parameters collected
third_params = [hp_lr, hp_knn, hp_lin_svm, hp_lda, hp_qda, hp_rf, hp_grad_boost, hp_extra]

# sklearn models
third_clf = [LogisticRegression(solver="saga", random_state=rs), 
          KNeighborsClassifier(), 
          LinearSVC(dual="auto", random_state=rs), 
          LinearDiscriminantAnalysis(solver="lsqr"), 
          QuadraticDiscriminantAnalysis(),
          RandomForestClassifier(random_state=rs), 
          GradientBoostingClassifier(random_state=rs), 
          ExtraTreesClassifier(random_state=rs)]

# feature mode for model
third_feat_mode = ["numeric", 
                    "numeric", 
                    "numeric", 
                    "numeric", 
                    "numeric", 
                    "numeric", 
                    "numeric", 
                    "numeric"]

third_zip = zip(third_clf_names, third_clf, third_params, third_feat_mode)
print(f"length of clf_names {len(third_clf_names)} & clf {len(third_clf)} & parameters {len(third_params)} & feat_mode {len(third_feat_mode)}")

length of clf_names 8 & clf 8 & parameters 8 & feat_mode 8


In [31]:
third_exp = "PSP_3a_HP-tune_w/o_class_weights"
third_info = "3. Training of models WITH hyperparameter tuning & w/ REDUCED features set "
"""hp_tune_model(search_mode="random", 
              zip_iter=third_zip, 
              x_train_raw=x_train_raw_light, x_train_numerical=x_train_numerical_light, 
              x_test_raw=x_test_raw_light, x_test_numerical=x_test_numerical_light, 
              y_train=y_train, y_test=y_test, 
              features_numerical=features_light, 
              exp_name=third_exp, info_text=third_info)"""

Training and logging of model Logistic Regression as classifier LogisticRegression(random_state=23, solver='saga') with feature mode numeric
Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END ...C=1.4100000000000001, l1_ratio=0.9, penalty=None; total time=   0.1s
[CV] END ...C=1.4100000000000001, l1_ratio=0.9, penalty=None; total time=   0.1s
[CV] END ...C=1.4100000000000001, l1_ratio=0.9, penalty=None; total time=   0.1s
[CV] END ...C=1.4100000000000001, l1_ratio=0.9, penalty=None; total time=   0.1s
[CV] END ...C=1.4100000000000001, l1_ratio=0.9, penalty=None; total time=   0.1s
[CV] END ...........C=1.81, l1_ratio=0.6, penalty=elasticnet; total time=   0.1s
[CV] END ...........C=1.81, l1_ratio=0.6, penalty=elasticnet; total time=   0.1s
[CV] END ...........C=1.81, l1_ratio=0.6, penalty=elasticnet; total time=   0.1s
[CV] END ...........C=1.81, l1_ratio=0.6, penalty=elasticnet; total time=   0.1s
[CV] END ...........C=1.81, l1_ratio=0.6, penalty=elasticnet; total 

KeyboardInterrupt: 

# Modeling no. 4: Fine-tuning of Top3-models via GridSearch

In [33]:
# collect all models
# names
fourth_clf_names = ["Random Forest", "Extra Trees", "Gradient Boosting"]

# configure parameters per model
hp_fine_rf = {"n_estimators": np.arange(150, 250, 10), 
         "max_depth": np.arange(15, 25, 1), 
         "min_samples_split": np.arange(2, 10, 2), 
         "min_samples_leaf": np.arange(25, 35, 1)}
hp_fine_grad_boost = {"learning_rate":np.arange(0.3, 0.7, 0.1), 
                 "subsample": np.arange(0.4, 0.99, 0.1), 
                 "min_samples_split": np.arange(40, 50, 1), 
                 "min_samples_leaf": np.arange(22, 32, 1), 
                 "max_depth": np.arange(2, 6, 1)}
hp_fine_extra = {"max_depth": np.arange(15, 25, 1), 
            "min_samples_split": np.arange(35, 45, 1), 
            "min_samples_leaf": np.arange(20, 30, 1), 
            "min_weight_fraction_leaf": np.arange(0, 0.3, 0.1)}

# parameters collected
fourth_params = [hp_fine_rf, hp_fine_extra, hp_fine_grad_boost]

# sklearn models
fourth_clf = [RandomForestClassifier(criterion="entropy", max_features="log2", random_state=rs), 
              ExtraTreesClassifier(criterion="entropy", max_features="sqrt", bootstrap=False, random_state=rs), 
              GradientBoostingClassifier(loss="log_loss", criterion="squared_error", max_features="log2", random_state=rs)]

# feature mode for model
fourth_feat_mode = ["numeric", "numeric", "numeric"]

fourth_zip = zip(fourth_clf_names, fourth_clf, fourth_params, fourth_feat_mode)
print(f"length of clf_names {len(fourth_clf_names)} & clf {len(fourth_clf)} & parameters {len(fourth_params)} & feat_mode {len(fourth_feat_mode)}")

length of clf_names 3 & clf 3 & parameters 3 & feat_mode 3


In [34]:
fourth_exp = "PSP_4a_finetune_w/o_class_weights"
fourth_info = "4. Training of models WITH hyperparameter tuning & w/ REDUCED features set "
hp_tune_model(search_mode="grid", 
              zip_iter=fourth_zip, 
              x_train_raw=x_train_raw_light, x_train_numerical=x_train_numerical_light, 
              x_test_raw=x_test_raw_light, x_test_numerical=x_test_numerical_light, 
              y_train=y_train, y_test=y_test, 
              features_numerical=features_light, 
              exp_name=fourth_exp, info_text=fourth_info)

2024/05/01 20:19:07 INFO mlflow.tracking.fluent: Experiment with name 'PSP_4_finetune_w/o_class_weights' does not exist. Creating a new experiment.


Training and logging of model Random Forest as classifier RandomForestClassifier(random_state=23) with feature mode numeric
Fitting 5 folds for each of 36000 candidates, totalling 180000 fits
[CV] END criterion=gini, max_depth=15, max_features=None, min_samples_leaf=25, min_samples_split=2, n_estimators=150; total time=   5.5s
[CV] END criterion=gini, max_depth=15, max_features=None, min_samples_leaf=25, min_samples_split=2, n_estimators=150; total time=   5.3s
[CV] END criterion=gini, max_depth=15, max_features=None, min_samples_leaf=25, min_samples_split=2, n_estimators=150; total time=   5.3s
[CV] END criterion=gini, max_depth=15, max_features=None, min_samples_leaf=25, min_samples_split=2, n_estimators=150; total time=   6.5s
[CV] END criterion=gini, max_depth=15, max_features=None, min_samples_leaf=25, min_samples_split=2, n_estimators=150; total time=   6.7s
[CV] END criterion=gini, max_depth=15, max_features=None, min_samples_leaf=25, min_samples_split=2, n_estimators=160; total

KeyboardInterrupt: 